<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/CART_classification_worldwide_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=TctV5Gh9X3H2QGlDJw9kR6r4-Q7FYuqqPE_B6yRjH-I&tc=gEglmldV5eg26VgZxiqhl_cen0MuZgBaszm7dKqA3c8&cc=EhhwA4wHay9SbvcHWOru2G87wR2j1_tKRAtVdRGZwIo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlN71Xx002FdS-4CPyM3qCmaC9wCF65n1uTx9zodMthzfSVlKt3JxM

Successfully saved authorization token.


In [30]:
# Input imagery is a cloud free Landsat 8 composite
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1")

image = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': l8.filterDate('2020-01-01', '2020-12-31'),
     'asFloat': True
})

In [31]:
# Use these bands for prediction
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B10', 'B11']

In [32]:
# Load training points. The numeric property 'class' stores known labeles.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

In [33]:
# This property of the table stores the land cover labels
label = 'landcover'

In [34]:
#Overlay the points on the image to get training
training = image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 30
})

In [35]:
#Train a CART classifier with default parameters
trained = ee.Classifier.smileCart().train(training, label, bands)

In [36]:
# Classify the image with the same bands used for training
classified = image.select(bands).classify(trained)

In [37]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[25, 29], zoom_start = 6)

# Add the layer to the map object.
my_map.add_ee_layer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
my_map.add_ee_layer(classified, {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
             'classification')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)